# Lecture 5A: MapReduce
## Introduction to the paradigm
### Martijn Herber Hanzehogeschool Groningen

# MapReduce is a Programming Paradigm
## What's a paradigm?
* You have experience with "custom" parallellization...
* ...but it's a lot of hard work
* What if setting up your computation could be easier?
    * Automatic, even?
* That requires a new way of structuring your problem and data... Paradigm shift!

# So... what's the paradigm?
* _IF_ you can structure your data into equal units...
* ...that you can apply an operation to... the Map()
* ...Without interdependencies...
* ...and you can combine the results using one other operation... the Reduce()
* MapReduce can take these operations and take care of the rest

# In pseudocode:
## Input & Output: each a set of key/value pairs

## map(in_key, in_value) -> list(out_key, intermediate_value)
* Processes input key/value pair
* Produces set of intermediate pairs
## reduce(out_key, list(intermediate_value)) -> list(out_value)
* Combines all intermediate values of a particular key
* Produces a set of merged output values (perhaps just one)

# Mapping
* Mapping is simply applying a function to the data.
* The data can be simple (a string) or complex (an object) but it needs to be _homogeneous_
* It is associated with a key, which may or may not be useful
    * Eg. a filename or line number is not used further
    * A word literal or codon is
* The data needs to have no interdependencies
    * I.e. processing file 1 doesn't require info from file 2's calculation

# Reducing
* The reduce function is called on completed mappings
* It may do nothing!
    * except return the data to the calling program!
* It tends to aggregate data based on the mapping's key
    * I.e. word frequency
* It is scheduled _separately_ from mapping
    * And uses a different datastore

# What kind of tasks?
* Word Count
* Word Histogram
* Pattern Search (grep)
* Reverse index / Reverse Graph
* Sorting
* Matrix Multiplication

## What can you think of?
* For next lecture: 
    * Read the original MapReduce Paper by Ghemawat et al
    * Think of a job in the context of Omics 2 that fits the MapReduce Paradigm
    * (In 1 slide) Present this job

In [1]:
import json
import sys 

class MapReduce:
    def __init__(self):
        self.intermediate = {}
        self.result = []

    def emit_intermediate(self, key, value):
        self.intermediate.setdefault(key, [])
        self.intermediate[key].append(value)

    def emit(self, value):
        self.result.append(value) 

    def execute(self, data, mapper, reducer):
        for line in data:
            record = json.loads(line)
            mapper(record)

        for key in self.intermediate:
            reducer(key, self.intermediate[key])
        
        #this should be overwritten with the actual action to do with the output
        for item in self.result:
            pass
            # print, write or do something to output

In [ ]:
def mapper(record):
    value = 0 # some map function of record
    mr.emit_intermediate(value, 1) #emit the intermediate value 

def reducer(key, list_of_values):
    result = 0 # some reduce function
    mr.emit((result)) #emit the result
    
## Basic usage example
mr = MapReduce()
mr.execute('path_to_data', mapper, reducer)

# Map reduce phases
<img src="phases.png">

# Seminal 2004 Paper by Dean & Ghemawat (Google)
<img src="mapreduce_system.png">
In code the fundamental operations are:

# Implementation
* There is one master node
* Master partitions input file into M splits, by key
* Master assings workers to the M map tasks, keeps tabs on them
* Workers write their output to local disk, partition into R regions
* Master assigns workers to the R reduce tasks
* Reduce workers read regions from the map worker's disks

# Hadoop: An open source implementation
<img src="hadoop.png">

# Our implementation
* We will base it on Assignment 3
* We will use the filesystem for data, Queue's for tasks
* We will pass the functions around generically
* We will use the generic iterator interface to iterate over data
* The Server will dole out tasks in multiple Queue's

# Iterators
## You can make anything behave like an iterable by returning an _iterator_
## Thanks to Python's Duck Typing ("If it quack's like a duck...") there is no semantic or syntactic difference
## Just implement the __iter__ and __next__ methods.

In [5]:
import re, reprlib
RE_WORD = re.compile('\w+')

In [11]:
class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = re.compile('\w+').findall(text)
        
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return SentenceIterator(self.words)

    def __len__(self):
        return len(self.words)

In [7]:
class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
            
        self.index += 1
        return word
    
    def __iter__(self):
        return self

In [12]:
s = Sentence('"Four score and seven years ago our fathers brought forth on this continent, a new nation, conceived in Liberty, and dedicated to the proposition that all men are created equal." said Old Abe')
for word in s:
    print(word)


Four
score
and
seven
years
ago
our
fathers
brought
forth
on
this
continent
a
new
nation
conceived
in
Liberty
and
dedicated
to
the
proposition
that
all
men
are
created
equal
said
Old
Abe


In [13]:
list(s)

['Four',
 'score',
 'and',
 'seven',
 'years',
 'ago',
 'our',
 'fathers',
 'brought',
 'forth',
 'on',
 'this',
 'continent',
 'a',
 'new',
 'nation',
 'conceived',
 'in',
 'Liberty',
 'and',
 'dedicated',
 'to',
 'the',
 'proposition',
 'that',
 'all',
 'men',
 'are',
 'created',
 'equal',
 'said',
 'Old',
 'Abe']